# Segmenting and Clustering Neighborhoods in Toronto

#### Scraping the data from the Wikipedia and creating neighborhood table 

In [9]:
!pip install beautifulsoup4 #install beautifulsoup4

     |████████████████████████████████| 112kB 24.4MB/s eta 0:00:01


In [28]:
!pip install lxml

     |████████████████████████████████| 5.8MB 9.0MB/s eta 0:00:01


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml.html as lh

In [16]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [17]:
soup = BeautifulSoup(url.content, 'html.parser')

In [25]:
n_table = soup.find('table', class_='wikitable')
df = pd.read_html(str(n_table))
T_df = pd.DataFrame(df[0])

In [48]:
print(T_df.shape)
T_df.head(10)

(287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [36]:
dr_df = T_df.drop(T_df[T_df['Borough']=='Not assigned'].index)

In [47]:
dr_df.head(20)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [109]:
dr_df[dr_df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Not assigned


In [113]:
n_df = dr_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)

In [111]:
n_df

Postcode  Borough    
M1B       Scarborough                                     [Rouge, Malvern]
M1C       Scarborough             [Highland Creek, Rouge Hill, Port Union]
M1E       Scarborough                  [Guildwood, Morningside, West Hill]
M1G       Scarborough                                             [Woburn]
M1H       Scarborough                                          [Cedarbrae]
                                               ...                        
M9N       York                                                    [Weston]
M9P       Etobicoke                                            [Westmount]
M9R       Etobicoke      [Kingsview Village, Martin Grove Gardens, Rich...
M9V       Etobicoke      [Albion Gardens, Beaumond Heights, Humbergate,...
M9W       Etobicoke                                            [Northwest]
Name: Neighbourhood, Length: 103, dtype: object

In [114]:
n_df = n_df.reset_index()
n_df['Neighbourhood'] = n_df['Neighbourhood'].str.join(', ')
n_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [118]:
#n_df[n_df[n_df['Neighbourhood']=='Not assigned'].index]
n_df.loc[(n_df.Neighbourhood=='Not assigned'), 'Neighbourhood'] = n_df.loc[(n_df.Neighbourhood=='Not assigned'), 'Borough']

In [120]:
n_df[n_df['Borough']=='Queen\'s Park']

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park
